# Adaptive PDE discretizations on cartesian grids 
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : The wave equation

We use a novel discretization of the Dirichlet elastic energy to solve the elastic wave equation, and compare the first arrival times with those computed in the high frequency asymptotic using an eikonal equation.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticityDirichlet','NonDiv'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics.Seismic import Hooke
from agd import Metrics
from agd import AutomaticDifferentiation as ad
from agd import Domain
from agd.Plotting import savefig; #savefig.dirName = 'Images/ElasticityDirichlet'
norm_infinity = ad.Optimization.norm_infinity

from agd.ExportedCode.Notebooks_Div.ElasticEnergy import ElasticEnergy

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse; import scipy.sparse.linalg

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global Hooke
    Hooke, = rreload((Hooke,),rootdir="../..")

## 1. Model analysis


### 1.1 The Hamiltonian


The Hamiltonian of the elastic wave equation is separable, and is the sum of two quadratic terms: the kinetic energy, and the elastic energy.
$$
    H(v,m) = \frac 1 2 \int_\Omega c(\epsilon,\epsilon) + \frac{\|m\|^2}{\rho},
$$
where $c$ is the Hooke tensor, $\epsilon$ is the stress tensor, $m$ is the momentum density, and $\rho$ is the material density.
The finite differences approximation of $c(\epsilon,\epsilon)$ is the subject of [another notebook](ElasticEnergy.ipynb). In order to complete our Hamiltonian, we thus only need to discretize the kinetic energy.

In [5]:
def KineticEnergy(m,ρ):
    """Squared norm of the momentum, divided by the density, 
    which is (twice) the kinetic energy density."""
    return (m**2).sum(axis=0) / ρ

Based on the earlier discussion along the [discretization of $c(\epsilon,\epsilon)$](ElasticEnergy.ipynb), and on the [wave equation CFL condition](Time1D_Div.ipynb), we obtain the CFL condition for the wave equation. 

In [6]:
def CFL(dx,hooke,ρ,order=1):
    """Largest time step expected to be stable for the elastic wave equation"""
    tr = lp.trace(hooke.to_Mandel())
    L = (16,78)[order] # See elastic energy notebook
    return 2*dx / np.sqrt(L * ρ * tr)

### 1.2 Choosing a material

We collected a few examples of material elasticity tensors and density, for illustration purposes. For simplicity, we use a constant material, and a periodic domain.

In [7]:
# Constant hooke tensor over the domain
mica,ρ = Hooke.mica # Note ρ is in g/cm^3, which is inconsistent with the Hooke tensor SI units 
hooke = mica.extract_xz().rotate_by(0.5)

For the purposes of illustration, we also consider a simpler isotropic material, defined from its Lame coefficients.

In [8]:
hooke_iso = Hooke.from_Lame(1.,1.)

We are now in position to introduce the full hamiltonian, for a periodic domain at least. 

<!---
The following does not work, because AD simplification must occur before summation
--->

<!---
def Hamiltonian(hooke,ρ,dom,**kwargs):
    """Returns the Hamiltonian of the linear elastic wave equation."""
    Hq = lambda q: 0.5 * ElasticEnergy(q,hooke,dom,**kwargs).sum() * h**2 
    Hp = lambda p: 0.5 * KineticEnergy(p,ρ).sum() * h**2

    H = Metrics.Hamiltonian( (Hq, Hp), shape_free = X.shape, inv_inner=h**-2 )
#    H.separable_quadratic_set_sparse_matrices(simplify_ad=2)
    return H
--->

In [9]:
def Hamiltonian_Fct(hooke,ρ,dom,order=1):
    """Returns the Hamiltonian of the linear elastic wave equation."""
    Hq = lambda q: 0.5 * ElasticEnergy(q,hooke,dom,order=order).sum() * h**2 
    Hp = lambda p: 0.5 * KineticEnergy(p,ρ).sum() * h**2
    return  Metrics.Hamiltonian( (Hq, Hp), shape_free = X.shape, inv_inner=h**-2 )

### 1.3 Traveling waves

Traveling waves are eigenmodes of the wave equation. In the case of elastic materials, several waves of different velocities may travel in the same direction. The fastest is known as the *pressure wave*. There also exists two independent shear waves in three dimension, and a single one in two dimensions.

Traveling waves take the form
$$
    V \exp(k\cdot x - \omega t),
$$
where $k$ is the wave number, $\omega$ is the pulsation, and $V$ is the amplitude. Given the wave vector $k$, there exists $d$ possible pulsations, and $d$ possible amplitudes (up to multiplication by a scalar).

In [10]:
def wave(t,x,ω,k,V):
    t,x,ω,k,V = fd.common_field((t,x,ω,k,V),depths=(0,1,0,1,1))
    return V*np.exp(lp.dot_VV(k,x)-ω*t)

The `Hooke` class provides a method for computing the (normalized) amplitude and pulsation associated to a wave vector.

In [11]:
ReloadPackages()
mica,ρ = Hooke.mica
hooke = mica.extract_xz().rotate_by(0.5)
hooke_iso = Hooke.from_Lame(1.,1.)

In [12]:
k = [0.6,0.8]
ω,V = hooke_iso.waves(k,1.)

In dimension two, there are two wave modes. The fastest mode, associated with the largest pulsation, is the pressure wave.

In [13]:
print(f"Wave pulsations {ω} for wave vector {k}")

Wave pulsations [1.         1.73205081] for wave vector [0.6, 0.8]


In isotropic materials, the amplitude of the pressure wave is collinear with the wave vector, whereas the amplitude of shear waves is orthogonal to the wave vector.

In [14]:
print(f"Amplitude of the shear wave: {V[:,0]}, and pressure wave: {V[:,1]}")

Amplitude of the shear wave: [-0.8  0.6], and pressure wave: [0.6 0.8]


This property is lost in anisotropic materials. The amplitude of the shear and pressure waves is neither collinear nor orthogonal to the wave vector.

In [15]:
k = [0.6,0.8]
ω,V = hooke.waves(k,ρ)
print(f"Amplitude of the shear wave: {V[:,0]}, and pressure wave: {V[:,1]}, for wave vector {k}.")

Amplitude of the shear wave: [ 0.54538986 -0.8381825 ], and pressure wave: [-0.8381825  -0.54538986], for wave vector [0.6, 0.8].


Let us check, using automatic differentiation, the we do obtain eigenmodes of the wave equation, in the case of a constant hooke tensor and density.

In [16]:
def WaveModes(k,hooke,ρ):
    """Returns the wave modes associated with a given Hooke tensor and wave vector"""
    def wave_(ω,V): return lambda t,x : wave(t,x,ω,k,V)
    ω_,V_ = hooke.waves(k,ρ)
    return [wave_(ω,V) for ω,V in zip(ω_,V_.T)]

In [17]:
def WaveResidue(t,x,wave,hooke,ρ):
    # Differentiate twice w.r.t time
    t_ad = ad.Dense2.identity(constant=t)
    w_tt = wave(t_ad,x).hessian(0,0)
    
    # Differentiate twice w.r.t position
    x_ad = ad.Dense2.identity(constant=x)
    w_xx = wave(t,x_ad).hessian() # Axes : partial derivatives, then coordinates.
    
    # Contract with the Hooke tensor
    d = hooke.vdim
    hk = hooke.to_depth4() # Format hooke tensor as a depth 4 tensor
    Δw = sum(hk[:,i,j,k]*w_xx[i,j,k] for i in range(d) for j in range(d) for k in range(d))

    return ρ*w_tt - Δw

In [18]:
t,x = 1.,[2.,3.]; material = (hooke_iso,1.)
shearW,pressW = WaveModes(k,*material)
assert np.allclose(WaveResidue(t,x,shearW,*material), 0.)
assert np.allclose(WaveResidue(t,x,pressW,*material), 0.)

In [19]:
t,x = 1.,[2.,3.]; material = (hooke,ρ)
shearW,pressW = WaveModes(k,*material)
assert np.allclose(WaveResidue(t,x,shearW,*material), 0.)
assert np.allclose(WaveResidue(t,x,pressW,*material), 0.)

## 2. Constant material

We compute elastic waves in a constant material, either isotropic or crystalline, in a periodic domain.

In [20]:
# Periodic domain [-1,1]^2
aX,dx = np.linspace(-1,1,endpoint=False,retstep=True)
X=np.array(np.meshgrid(aX,aX,indexing='ij'))
dom = Domain.MockDirichlet(X.shape[1:],dx,padding=None) #Periodic domain (wrap instead of pad)

In [26]:
def Hamiltonian(hooke,ρ,dom,order=1):
    """Returns the Hamiltonian of the linear elastic wave equation, with sparse matrices."""
    v_ad = ad.Sparse2.identity((dom.vdim,*dom.shape))
    Elastic_ad = 0.5 * ElasticEnergy(v_ad,hooke,dom,order=order)
    for i in range(2): Elastic_ad.simplify_ad() # These intermediary steps cannot be done by Hamiltonian class
    h = dom.gridscale
    Hq_ad = 0.5 * Elastic_ad.sum() * h**2
    Hq_spmat = scipy.sparse.coo_matrix(Hq_ad.triplets()).tocsc()

    Hp = lambda p: 0.5 * KineticEnergy(p,ρ).sum() * h**2
    H = Metrics.Hamiltonian( (Hq_spmat, Hp), shape_free = X.shape, inv_inner=h**-2 )
    H.separable_quadratic_set_sparse_matrices() # yields Hp_spmat
    return H

### 2.1 Isotropic 

First let us set the Hamiltonian, governing the wave equation, and the time step.

In [27]:
material = (hooke_iso,1.)
WaveH = Hamiltonian(*material,dom)
dt = CFL(dx,*material)

Then, we choose an initial condition, space periodic for simplicity.

In [28]:
k = [np.pi,0.] 
pressW,shearW = WaveModes(k,*material)

In [29]:
q0 = pressW(0,X) # Initial displacement
t_ad = ad.Dense.identity(constant=0.)
p0 = pressW(t_ad,X).gradient(0) # Initial momentum (rho=1)

In [34]:
p0.shape

(2, 50, 50)

In [36]:
class A: pass
class B(A): pass

In [39]:
np.prod((),dtype=int)

1

In [ ]:
# Need some flattening of the input

In [32]:
WaveH.H(q0,p0)

ValueError: could not interpret dimensions

In [31]:
niter=10
Q,P = WaveH.integrate(q0,p0, scheme='Euler-p',niter=niter,T=niter*dt,path=True)

ValueError: could not interpret dimensions

In [ ]:
help(Wave.integrate)